### 协作型过滤
通过对人群搜索，找出品味相近的一小群，筛选出推荐的列表。
### 相似度评价指标
* 欧氏距离

In [7]:
from critics import critics
from math import sqrt
def sim_distance(prefs,person1,person2):
    si={}
    #计算两个用户影评内容的各项的相关列表
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    if len(si)==0:
        return 0
    
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    #将计算结果映射到［０，１］之间，１为相似度极高
    return 1/(1+sqrt(sum_of_squares))

sim_distance(critics,'Lisa Rose','Gene Seymour')

0.29429805508554946

* 皮尔逊相关系数：

    判断两组数据与某一直线拟合程度的度量，取值$\pm 1$之间（正相关1 负相关-1 无关0）

    当数据出现夸大分值情况，可给出更准确的相似度量
    
    $p_{X,Y}=\frac{cov(X,Y)}{\sigma_X\sigma_Y}＝\frac{\sum{XY}-\frac{\sum X\sum Y}{N}}{\sqrt{(\sum X^2-\frac{(\sum X)^2}{N})(\sum Y^2-\frac{(\sum Y)^2}{N})}}$
    

In [10]:
def sim_pearson(prefs,p1,p2):
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item]=1
    if len(si)==0:
        return 0
    n=len(si)
    #偏好求和
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    sum1sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2sq=sum([pow(prefs[p2][it],2) for it in si])
    
    psum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    #皮尔逊值
    num=psum-(sum1*sum2/n)
    den=sqrt((sum1sq-pow(sum1,2)/n)*(sum2sq-pow(sum2,2)/n))
    if den==0:
        return 0
    
    r=num/den
    
    return r

sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

* Jaccard系数：

    两集合的差集与并集的比$J(A,B)=\frac{|A\cap B|}{|A\cup B|}=\frac{|A\cap B|}{|A|+|B|-|A\cap B|}$
    
    系数越大，相似度越高；元素取值仅能为0或1，信息不丰富

### 为用户打分

In [11]:
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    #输出相似度与用户名的元组，并取top n
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

topMatches(critics,'Toby',n=5)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig'),
 (0.66284898035987, 'Jack Matthews'),
 (0.38124642583151164, 'Gene Seymour')]

### 推荐物品
标准：用户的相似度乘影评得分并加和／相应用户的相似度之和

In [15]:
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simsum={}
    for other in prefs:
        if other==person:
            continue
        sim=similarity(prefs,person,other)
        if sim<=0:
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item]==0:
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                simsum.setdefault(item,0)
                simsum[item]+=sim
    ranking=[(total/simsum[item],item) for item,total in totals.items()]
    ranking.sort()
#     print totals
    ranking.reverse()
    return ranking

getRecommendations(critics,'Toby',similarity=sim_pearson)

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

### 推荐相似度相同的物品

In [31]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            result[item][person]=prefs[person][item]
    return result

film=transformPrefs(critics)
topMatches(film,'Just My Luck')

[(0.5555555555555556, 'The Night Listener'),
 (-0.3333333333333333, 'Snakes on a Plane'),
 (-0.42289003161103106, 'Superman Returns'),
 (-0.4856618642571827, 'You, Me and Dupree'),
 (-0.9449111825230676, 'Lady in the Water')]

In [32]:
getRecommendations(film,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

### 协作型过滤
* 基于用户：以相似的人群喜好推荐相应物品

* 基于物品：先计算好物品的相近物品，根据用户历史，推荐相应物品（计算近似物品步骤可预先进行，即物品变化相比用户变化更不频繁）

* 类型选择：基于物品会增加存储开销，但比基于用户更快；若数据集稀疏，基于物品的效果更好，若密集，则无差别；若是规模小，且变化频繁的内存数据集，则基于用户更优